In [3]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [4]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity.
            Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
            Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism.
            Your job is to make the email sound warm, confident, and human — while staying professional
            and appropriate for the audience. Improve the emotional resonance, polish the phrasing,
            and adjust any words that may come off as stiff, cold, or overly casual.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology,
            and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all
            prior agent responses and revisions, and then **synthesize the best ideas** into a unified,
            polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements;
            Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional,
            effective, and readable.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review
            of the synthesized email and determine if it meets professional standards. Review the email for:
            Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
            Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone,
            or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.""",
)

In [5]:

text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [6]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.
---------- TextMessage (ClarityAgent) ----------
Hi! I'm hungry. Please buy me lunch and invest in my business. Thanks.
---------- TextMessage (ToneAgent) ----------
Subject: A Quick Favor

Hi [Recipient's Name],

I hope this message finds you well! I wanted to reach out because I’m feeling a bit hungry today, and I thought it might be a great opportunity to catch up over lunch. If you’re available, I’d love to share some ideas about my business and discuss how you could potentially invest in its future.

Thank you for considering this! I truly appreciate your support.

Warm regards,

[Your Name]  
[Your Contact Information]  
---------- TextMessage (PersuasionAgent) ----------
Subject: Let’s Grab Lunch and Explore a Business Opportunity!

Hi [Recipient's Name],

I hope you’re doing well! I’m reaching out with a two-fold request that I believe can lead to exciting opportunities for b

TaskResult(messages=[TextMessage(id='8542a464-72df-45d7-8d50-3cc531b3e9ab', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 4, 2, 0, 25, 405248, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='aa70da34-df68-4213-b85a-f353e23a3e21', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=75, completion_tokens=17), metadata={}, created_at=datetime.datetime(2025, 10, 4, 2, 0, 29, 112430, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry. Please buy me lunch and invest in my business. Thanks.", type='TextMessage'), TextMessage(id='eb273421-891c-47eb-a58f-d2d062d3bf60', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=120, completion_tokens=101), metadata={}, created_at=datetime.datetime(2025, 10, 4, 2, 0, 32, 7009, tzinfo=datetime.timezone.utc), content="Subject: A Quick Favor\n\nHi [Recipient's Name],\n\nI hope this message finds you we